In [1]:
import numpy as np
import astropy.constants as const
import os


# single grain size from Dsharp opac dustopac.inp only 1 input block
# make a flow chart
#

In [18]:
#############
# Constants #
#############
au   = const.au.cgs.value    # Astronomical Unit  [cm]
pc   = const.pc.cgs.value    # Parsec             [cm]
Msun = const.M_sun.cgs.value # Solar Mass         [g]
Lsun = const.L_sun.cgs.value # Solar Luminosity   [ergs/s]
Rsun = const.R_sun.cgs.value # Solar Radius       [cm]
Tsun = 5780                  # Solar Temperature  [K] 
pi   = np.pi                 # circle stuff lol   [radians]

##########################
# radial cell wall setup #
##########################
rin_au         = 20             # inner radius [am]                      # Disk Param
rout_au        = 110             # outer radius [am]                      # Disk Param  
rin            = rin_au * au     # inner radius [cm]   
rout           = rout_au * au    # outer radius [cm]
r_factor       = 5               # point where to change grid style       # Grid Param
rchange        = r_factor * rin  # radius where grid changes from [au]    # Grid Param
nr_inner_cells = 10              # number of cell for inner FINE region   # Grid Param
nr_outer_cells = 10              # number of cell for outer COARSE region # Grid Param

r_grid_params = [                                             
    [rin, rchange, nr_inner_cells + 1, False],      # inner radius args
    [*np.log10([rchange, rout]), nr_outer_cells + 1] # outer radius args   
]

# to avoid duplication, endpoint is set to FALSE where rchange 
# will not be accounted for in inner grid but recovered in outer

r_inner   = np.linspace(*r_grid_params[0])     # inner FINE grid style
r_outer   = np.logspace(*r_grid_params[1])     # outer COARSE grid style
r_grid    = np.concatenate([r_inner, r_outer]) # merge the two grids


####################################
# Theta cell wall setup (altitude) #
####################################

theta_min         = 0      # minimum altitude                   # grid param
theta_change      = pi / 3 # loacation where grid style changes # grid param
theta_midplane    = pi / 2 # mid-plane location
ntheta_cells_high = 20      # number of cells near the plane     # grid param
ntheta_cells_low  = 20      # number of cells near the plane     # grid param

theta_grid_params = [
    [theta_min, theta_change, ntheta_cells_high + 1, False], # theta high args
    [theta_change, theta_midplane, ntheta_cells_low + 1]     # theta low args
]

theta_grid_high     = np.linspace(*theta_grid_params[0])                         # COARSE grid spacing
theta_grid_low      = np.linspace(*theta_grid_params[1])                         # FINE grid spacing near the mid-plane
theta_grid_1st_half = np.concatenate([theta_grid_high, theta_grid_low])          # 0 - 90 degress
theta_grid_2nd_half = np.flip(pi - theta_grid_1st_half, 0)[1:]                   # from 90 to 180 degress
theta_grid          = np. concatenate([theta_grid_1st_half,theta_grid_2nd_half]) # 


#################################
# Phi cell wall setup (azimuth) #
#################################

nphi_cells        = 10                                   # number of cells in polar axis # Grid param
phi_grid          = np.linspace(0, 2*pi, nphi_cells + 1) # polar angles

##############################
# Find centers of cell walls #
##############################

nr     = len(r_grid)
nphi   = len(phi_grid)
ntheta = len(theta_grid)

r_center_grid     = 0.5 * ( r_grid[0:nr-1] + r_grid[1:nr] )
phi_center_grid   = 0.5 * ( phi_grid[0:nphi-1] + phi_grid[1:nphi] )
theta_center_grid = 0.5 * ( theta_grid[0:ntheta-1] + theta_grid[1:ntheta] )

nr_center     = len(r_center_grid)
nphi_center   = len(phi_center_grid)
ntheta_center = len(theta_center_grid)

#print(nr, nr_center)
#print(nphi, nphi_center)
#print(ntheta, ntheta_center)

###############################
# Write the spatial grid file #
###############################

def writeSpatialGridFile():
    print('Writing amr_grid.inp ...')
    with open('amr_grid.inp', 'w+') as f:
        # Write formating section
        f.write('1\n')                              # iformat
        f.write('0\n')                              # AMR grid style  (0=regular grid, no AMR)
        f.write('100\n')                            # Coordinate system
        f.write('0\n')                              # grid info
        f.write('1 1 1\n')                          # include coordinate: r, phi, theta   #
        f.write('{} {} {}\n'.format(nr_center,      # grid  cell info
                                    nphi_center, 
                                    ntheta_center)) 

        # Write grid values
        for r in r_grid_cm:
            f.write('{} '.format(r))
        f.write('\n')
        for phi in phi_grid:
            f.write('{} '.format(phi))
        f.write('\n')
        for theta in theta_grid:
            f.write('{} '.format(theta))
            
print(phi_center_grid)
print(theta_center_grid)
print(r_center_grid)

[0.31415927 0.9424778  1.57079633 2.19911486 2.82743339 3.45575192
 4.08407045 4.71238898 5.34070751 5.96902604]
[0.02493328 0.07479983 0.12466638 0.17453293 0.22439948 0.27426603
 0.32413258 0.37399913 0.42386568 0.47373223 0.52359878 0.57346533
 0.62333188 0.67319843 0.72306498 0.77293153 0.82279808 0.87266463
 0.92253118 0.97239773 1.02226428 1.06028752 1.08646746 1.1126474
 1.13882734 1.16500728 1.19118721 1.21736715 1.24354709 1.26972703
 1.29590697 1.32208691 1.34826685 1.37444679 1.40062672 1.42680666
 1.4529866  1.47916654 1.50534648 1.53152642 1.55770636 1.5838863
 1.61006623 1.63624617 1.66242611 1.68860605 1.71478599 1.74096593
 1.76714587 1.79332581 1.81950575 1.84568568 1.87186562 1.89804556
 1.9242255  1.95040544 1.97658538 2.00276532 2.02894526 2.05512519
 2.08130513 2.11932838 2.16919493 2.21906148 2.26892803 2.31879458
 2.36866113 2.41852768 2.46839423 2.51826078 2.56812733 2.61799388
 2.66786043 2.71772698 2.76759353 2.81746008 2.86732663 2.91719318
 2.96705973 3.0169

In [19]:
###################################################################
# Dust Density Profile
###################################################################

# Constants
rc_au      = 100   # Disk Parameter
Mdust_Msun = 1e-4  # Disk Parameter
gamma      = 1     # Disk Parameter
H100au     = 10    # Disk Parameter
beta       = 2     # Disk Parameter


def coordTransformation(r_sph, theta):
    # convert from spherical to cylindrical coordinates
    # inputs: 
    r_cylindrical = r * np.sin(theta)
    z_cylindrical = r * np.cos(theta)
    return r_cylindrical , z_cylindrical

def sigma_norm(Mdust, rin_au, rout_au, rc_au, gamma): 
    # compute the normalization constant linked total dust mass
    exponent = 2. - gamma
    density_norm = (
        Mdust * (gamma - 2.) / (2. * pi * rc * rc) / 
        (
            np.exp(-(rout_au / rc_au)**exponent) - 
            np.exp(-(rin_au / rc_au)**exponent)
        )
    )
    return density_norm     
    

# reformat funciton (nasty readability)
def volumeDensity(r, z, rc, Mdust, sigma0, H100au, gamma, beta):
    # Calculate the volume dust density
    # inputs: radius [cm], z-component [cm], disk mass [g], gamma "radial
    # profile" index, the scale height at 100 au [cm], beta "flaring
    # parameter" index
    
    def H_r():
        # Scale height
        # Calculate the scale height at some radius
        return H100au * au * ( r / ( 100 * au ) )**beta    
    
    def Sigma_r():
        # surface density in cgs
        return sigma0*( ( r / rc )**( -gamma ) ) * np.exp( - ( r / rc )**( 2 - gamma ) )
    
    H = H_r()
    Sigma = Sigma_r()
    rho = (
        Sigma / ( np.sqrt(2 * pi) * H ) * 
        np.exp( - 0.5 * ( z * z / ( H * H ) ) ) 
    )
    return rho

rc           = rc_au * au 
Mdust        = Mdust_Msun * Msun
sigma0       = sigma_norm(Mdust = Mdust, rin_au = rin_au, rout_au = rout_au, 
                          rc_au = rc_au, gamma = gamma)

# check rc with rin or rout:
if rin > rc or rout < rc:
    raise ValueError('If rc < rin or rc > rout : volume density calculation will result in negative number')
    

density_grid = np.zeros((nphi_center, ntheta_center, nr_center))

for i in range(nphi_center):
    for j, t in enumerate(theta_center_grid):
        for k, r in enumerate(r_center_grid): 
            r_cyl , z_cyl = coordTransformation(r_sph = r, theta = t)
            if r_cyl < rin or r_cyl > rout:
                density_rho = 0
                density_grid[i][j,k] = density_rho
                print('density[{}, {}, {}, ispec=1] = {}'.format(k+1,j+1,i+1,density_rho)) # print check for appropiate file stuctture
            else:
                density_rho = volumeDensity(r = r_cyl, z = z_cyl, rc = rc, Mdust = Mdust, sigma0 = sigma0,
                                            gamma = gamma, H100au = H100au, beta = beta)
                density_grid[i][j,k] = density_rho
                print('density[{}, {}, {}, ispec=1] = {}'.format(k+1,j+1,i+1,density_rho)) # print check for appropiate file stuctture
            
#r_cyl, z_cyl = coordTransformation(radius_grid_cm, theta_grid)





density[1, 1, 1, ispec=1] = 0
density[2, 1, 1, ispec=1] = 0
density[3, 1, 1, ispec=1] = 0
density[4, 1, 1, ispec=1] = 0
density[5, 1, 1, ispec=1] = 0
density[6, 1, 1, ispec=1] = 0
density[7, 1, 1, ispec=1] = 0
density[8, 1, 1, ispec=1] = 0
density[9, 1, 1, ispec=1] = 0
density[10, 1, 1, ispec=1] = 0
density[11, 1, 1, ispec=1] = 0
density[12, 1, 1, ispec=1] = 0
density[13, 1, 1, ispec=1] = 0
density[14, 1, 1, ispec=1] = 0
density[15, 1, 1, ispec=1] = 0
density[16, 1, 1, ispec=1] = 0
density[17, 1, 1, ispec=1] = 0
density[18, 1, 1, ispec=1] = 0
density[19, 1, 1, ispec=1] = 0
density[20, 1, 1, ispec=1] = 0
density[21, 1, 1, ispec=1] = 0
density[1, 2, 1, ispec=1] = 0
density[2, 2, 1, ispec=1] = 0
density[3, 2, 1, ispec=1] = 0
density[4, 2, 1, ispec=1] = 0
density[5, 2, 1, ispec=1] = 0
density[6, 2, 1, ispec=1] = 0
density[7, 2, 1, ispec=1] = 0
density[8, 2, 1, ispec=1] = 0
density[9, 2, 1, ispec=1] = 0
density[10, 2, 1, ispec=1] = 0
density[11, 2, 1, ispec=1] = 0
density[12, 2, 1, ispec=1]

density[21, 33, 1, ispec=1] = 2.3208065941036167e-18
density[1, 34, 1, ispec=1] = 5.090466942262381e-31
density[2, 34, 1, ispec=1] = 9.2148775899798e-25
density[3, 34, 1, ispec=1] = 7.594074734264109e-22
density[4, 34, 1, ispec=1] = 2.6668235493338482e-20
density[5, 34, 1, ispec=1] = 2.0508019285292193e-19
density[6, 34, 1, ispec=1] = 6.9857611767470975e-19
density[7, 34, 1, ispec=1] = 1.484599058770888e-18
density[8, 34, 1, ispec=1] = 2.3614906587733697e-18
density[9, 34, 1, ispec=1] = 3.1185940472905372e-18
density[10, 34, 1, ispec=1] = 3.6389168754862326e-18
density[11, 34, 1, ispec=1] = 3.901212578468155e-18
density[12, 34, 1, ispec=1] = 3.947839386645656e-18
density[13, 34, 1, ispec=1] = 3.949543305792949e-18
density[14, 34, 1, ispec=1] = 3.9480226803619926e-18
density[15, 34, 1, ispec=1] = 3.943331892118047e-18
density[16, 34, 1, ispec=1] = 3.935531747157778e-18
density[17, 34, 1, ispec=1] = 3.924689061825069e-18
density[18, 34, 1, ispec=1] = 3.910876241082611e-18
density[19, 34,

density[13, 21, 2, ispec=1] = 7.574052855798613e-28
density[14, 21, 2, ispec=1] = 1.1685866204525796e-27
density[15, 21, 2, ispec=1] = 1.786901315582779e-27
density[16, 21, 2, ispec=1] = 2.7084438188560772e-27
density[17, 21, 2, ispec=1] = 4.069957203868636e-27
density[18, 21, 2, ispec=1] = 6.064296031823146e-27
density[19, 21, 2, ispec=1] = 8.961071617346036e-27
density[20, 21, 2, ispec=1] = 1.3133959378222917e-26
density[21, 21, 2, ispec=1] = 1.9096485122203537e-26
density[1, 22, 2, ispec=1] = 5.287445909728081e-175
density[2, 22, 2, ispec=1] = 6.884119462838874e-109
density[3, 22, 2, ispec=1] = 6.468946239382595e-77
density[4, 22, 2, ispec=1] = 4.192161077410713e-59
density[5, 22, 2, ispec=1] = 3.3313557425148494e-48
density[6, 22, 2, ispec=1] = 4.517868335251337e-41
density[7, 22, 2, ispec=1] = 3.639530172867592e-36
density[8, 22, 2, ispec=1] = 1.1716405376211423e-32
density[9, 22, 2, ispec=1] = 4.5166670770156266e-30
density[10, 22, 2, ispec=1] = 4.066068521452576e-28
density[11, 

density[8, 23, 3, ispec=1] = 2.161027365377171e-30
density[9, 23, 3, ispec=1] = 3.4004673919178857e-28
density[10, 23, 3, ispec=1] = 1.541728945591666e-26
density[11, 23, 3, ispec=1] = 2.8989886830833257e-25
density[12, 23, 3, ispec=1] = 1.1351824068702558e-24
density[13, 23, 3, ispec=1] = 1.5222311340192227e-24
density[14, 23, 3, ispec=1] = 2.0283834032669528e-24
density[15, 23, 3, ispec=1] = 2.6861163935353988e-24
density[16, 23, 3, ispec=1] = 3.535532217572729e-24
density[17, 23, 3, ispec=1] = 4.6258215413036765e-24
density[18, 23, 3, ispec=1] = 6.0169309823986165e-24
density[19, 23, 3, ispec=1] = 7.781446354360248e-24
density[20, 23, 3, ispec=1] = 1.0006701167793338e-23
density[21, 23, 3, ispec=1] = 1.2797116376288211e-23
density[1, 24, 3, ispec=1] = 2.0285978188421317e-132
density[2, 24, 3, ispec=1] = 5.286461011991424e-84
density[3, 24, 3, ispec=1] = 1.2724067520271128e-60
density[4, 24, 3, ispec=1] = 1.278816843016413e-47
density[5, 24, 3, ispec=1] = 1.1121212915366149e-39
densi

density[16, 33, 4, ispec=1] = 2.2479958237374223e-18
density[17, 33, 4, ispec=1] = 2.266713275079355e-18
density[18, 33, 4, ispec=1] = 2.283361713344095e-18
density[19, 33, 4, ispec=1] = 2.2979295608367717e-18
density[20, 33, 4, ispec=1] = 2.3104111519184796e-18
density[21, 33, 4, ispec=1] = 2.3208065941036167e-18
density[1, 34, 4, ispec=1] = 5.090466942262381e-31
density[2, 34, 4, ispec=1] = 9.2148775899798e-25
density[3, 34, 4, ispec=1] = 7.594074734264109e-22
density[4, 34, 4, ispec=1] = 2.6668235493338482e-20
density[5, 34, 4, ispec=1] = 2.0508019285292193e-19
density[6, 34, 4, ispec=1] = 6.9857611767470975e-19
density[7, 34, 4, ispec=1] = 1.484599058770888e-18
density[8, 34, 4, ispec=1] = 2.3614906587733697e-18
density[9, 34, 4, ispec=1] = 3.1185940472905372e-18
density[10, 34, 4, ispec=1] = 3.6389168754862326e-18
density[11, 34, 4, ispec=1] = 3.901212578468155e-18
density[12, 34, 4, ispec=1] = 3.947839386645656e-18
density[13, 34, 4, ispec=1] = 3.949543305792949e-18
density[14, 3

...

density[15, 78, 5, ispec=1] = 0.0
density[16, 78, 5, ispec=1] = 0.0
density[17, 78, 5, ispec=1] = 0.0
density[18, 78, 5, ispec=1] = 0.0
density[19, 78, 5, ispec=1] = 0.0
density[20, 78, 5, ispec=1] = 0.0
density[21, 78, 5, ispec=1] = 0.0
density[1, 79, 5, ispec=1] = 0
density[2, 79, 5, ispec=1] = 0
density[3, 79, 5, ispec=1] = 0
density[4, 79, 5, ispec=1] = 0
density[5, 79, 5, ispec=1] = 0
density[6, 79, 5, ispec=1] = 0
density[7, 79, 5, ispec=1] = 0
density[8, 79, 5, ispec=1] = 0
density[9, 79, 5, ispec=1] = 0
density[10, 79, 5, ispec=1] = 0
density[11, 79, 5, ispec=1] = 0
density[12, 79, 5, ispec=1] = 0
density[13, 79, 5, ispec=1] = 0
density[14, 79, 5, ispec=1] = 0
density[15, 79, 5, ispec=1] = 0
density[16, 79, 5, ispec=1] = 0
density[17, 79, 5, ispec=1] = 0
density[18, 79, 5, ispec=1] = 0
density[19, 79, 5, ispec=1] = 0
density[20, 79, 5, ispec=1] = 0
density[21, 79, 5, ispec=1] = 0
density[1, 80, 5, ispec=1] = 0
density[2, 80, 5, ispec=1] = 0
density[3, 80, 5, ispec=1] = 0
densit

density[6, 2, 7, ispec=1] = 0
density[7, 2, 7, ispec=1] = 0
density[8, 2, 7, ispec=1] = 0
density[9, 2, 7, ispec=1] = 0
density[10, 2, 7, ispec=1] = 0
density[11, 2, 7, ispec=1] = 0
density[12, 2, 7, ispec=1] = 0
density[13, 2, 7, ispec=1] = 0
density[14, 2, 7, ispec=1] = 0
density[15, 2, 7, ispec=1] = 0
density[16, 2, 7, ispec=1] = 0
density[17, 2, 7, ispec=1] = 0
density[18, 2, 7, ispec=1] = 0
density[19, 2, 7, ispec=1] = 0
density[20, 2, 7, ispec=1] = 0
density[21, 2, 7, ispec=1] = 0
density[1, 3, 7, ispec=1] = 0
density[2, 3, 7, ispec=1] = 0
density[3, 3, 7, ispec=1] = 0
density[4, 3, 7, ispec=1] = 0
density[5, 3, 7, ispec=1] = 0
density[6, 3, 7, ispec=1] = 0
density[7, 3, 7, ispec=1] = 0
density[8, 3, 7, ispec=1] = 0
density[9, 3, 7, ispec=1] = 0
density[10, 3, 7, ispec=1] = 0
density[11, 3, 7, ispec=1] = 0
density[12, 3, 7, ispec=1] = 0
density[13, 3, 7, ispec=1] = 0
density[14, 3, 7, ispec=1] = 0
density[15, 3, 7, ispec=1] = 0
density[16, 3, 7, ispec=1] = 0
density[17, 3, 7, isp

density[2, 82, 7, ispec=1] = 0
density[3, 82, 7, ispec=1] = 0
density[4, 82, 7, ispec=1] = 0
density[5, 82, 7, ispec=1] = 0
density[6, 82, 7, ispec=1] = 0
density[7, 82, 7, ispec=1] = 0
density[8, 82, 7, ispec=1] = 0
density[9, 82, 7, ispec=1] = 0
density[10, 82, 7, ispec=1] = 0
density[11, 82, 7, ispec=1] = 0
density[12, 82, 7, ispec=1] = 0
density[13, 82, 7, ispec=1] = 0
density[14, 82, 7, ispec=1] = 0
density[15, 82, 7, ispec=1] = 0
density[16, 82, 7, ispec=1] = 0
density[17, 82, 7, ispec=1] = 0
density[18, 82, 7, ispec=1] = 0
density[19, 82, 7, ispec=1] = 0
density[20, 82, 7, ispec=1] = 0
density[21, 82, 7, ispec=1] = 0
density[1, 1, 8, ispec=1] = 0
density[2, 1, 8, ispec=1] = 0
density[3, 1, 8, ispec=1] = 0
density[4, 1, 8, ispec=1] = 0
density[5, 1, 8, ispec=1] = 0
density[6, 1, 8, ispec=1] = 0
density[7, 1, 8, ispec=1] = 0
density[8, 1, 8, ispec=1] = 0
density[9, 1, 8, ispec=1] = 0
density[10, 1, 8, ispec=1] = 0
density[11, 1, 8, ispec=1] = 0
density[12, 1, 8, ispec=1] = 0
densi

density[5, 79, 8, ispec=1] = 0
density[6, 79, 8, ispec=1] = 0
density[7, 79, 8, ispec=1] = 0
density[8, 79, 8, ispec=1] = 0
density[9, 79, 8, ispec=1] = 0
density[10, 79, 8, ispec=1] = 0
density[11, 79, 8, ispec=1] = 0
density[12, 79, 8, ispec=1] = 0
density[13, 79, 8, ispec=1] = 0
density[14, 79, 8, ispec=1] = 0
density[15, 79, 8, ispec=1] = 0
density[16, 79, 8, ispec=1] = 0
density[17, 79, 8, ispec=1] = 0
density[18, 79, 8, ispec=1] = 0
density[19, 79, 8, ispec=1] = 0
density[20, 79, 8, ispec=1] = 0
density[21, 79, 8, ispec=1] = 0
density[1, 80, 8, ispec=1] = 0
density[2, 80, 8, ispec=1] = 0
density[3, 80, 8, ispec=1] = 0
density[4, 80, 8, ispec=1] = 0
density[5, 80, 8, ispec=1] = 0
density[6, 80, 8, ispec=1] = 0
density[7, 80, 8, ispec=1] = 0
density[8, 80, 8, ispec=1] = 0
density[9, 80, 8, ispec=1] = 0
density[10, 80, 8, ispec=1] = 0
density[11, 80, 8, ispec=1] = 0
density[12, 80, 8, ispec=1] = 0
density[13, 80, 8, ispec=1] = 0
density[14, 80, 8, ispec=1] = 0
density[15, 80, 8, isp

density[11, 68, 9, ispec=1] = 4.174273521371075e-85
density[12, 68, 9, ispec=1] = 1.1392995835463323e-79
density[13, 68, 9, ispec=1] = 1.707781687337495e-78
density[14, 68, 9, ispec=1] = 2.4305963448805484e-77
density[15, 68, 9, ispec=1] = 3.287787548898865e-76
density[16, 68, 9, ispec=1] = 4.2307892661173834e-75
density[17, 68, 9, ispec=1] = 5.1841103609480586e-74
density[18, 68, 9, ispec=1] = 6.054287158886025e-73
density[19, 68, 9, ispec=1] = 6.744983456643117e-72
density[20, 68, 9, ispec=1] = 7.174862588300717e-71
density[21, 68, 9, ispec=1] = 7.293559894854956e-70
density[1, 69, 9, ispec=1] = 0
density[2, 69, 9, ispec=1] = 0
density[3, 69, 9, ispec=1] = 0.0
density[4, 69, 9, ispec=1] = 0.0
density[5, 69, 9, ispec=1] = 0.0
density[6, 69, 9, ispec=1] = 1.0590451877796256e-259
density[7, 69, 9, ispec=1] = 5.726012457368772e-210
density[8, 69, 9, ispec=1] = 4.2340992341223e-174
density[9, 69, 9, ispec=1] = 2.169702640110692e-147
density[10, 69, 9, ispec=1] = 5.665896001454627e-127
den

In [31]:
print(density_grid)
print(density_grid.flatten())
a = density_grid.flatten()
print(a[500:600])

[[[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 ...

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]]
[0. 0. 0. 

In [53]:
#dim_array = [nphi, ntheta, nr]
dim_array = [4, 4, 4]
n_phi = 5
r = np.arange(1,111,10).reshape(1,-1)
theta = np.linspace(np.pi/2, np.pi,10).reshape(-1,1)
r_cyn = r * np.sin(theta)
print(r)
print(theta)
print(r_cyn, r_cyn.shape)




"""dDG = np.random.randn(*dim_array)

print(dDG)
print(dDG.shape)
dDG = dDG.flatten()
print(dDG)
print(dDG.shape)"""

[[  1  11  21  31  41  51  61  71  81  91 101]]
[[1.57079633]
 [1.74532925]
 [1.91986218]
 [2.0943951 ]
 [2.26892803]
 [2.44346095]
 [2.61799388]
 [2.7925268 ]
 [2.96705973]
 [3.14159265]]
[[1.00000000e+00 1.10000000e+01 2.10000000e+01 3.10000000e+01
  4.10000000e+01 5.10000000e+01 6.10000000e+01 7.10000000e+01
  8.10000000e+01 9.10000000e+01 1.01000000e+02]
 [9.84807753e-01 1.08328853e+01 2.06809628e+01 3.05290403e+01
  4.03771179e+01 5.02251954e+01 6.00732729e+01 6.99213505e+01
  7.97694280e+01 8.96175055e+01 9.94655831e+01]
 [9.39692621e-01 1.03366188e+01 1.97335450e+01 2.91304712e+01
  3.85273975e+01 4.79243237e+01 5.73212499e+01 6.67181761e+01
  7.61151023e+01 8.55120285e+01 9.49089547e+01]
 [8.66025404e-01 9.52627944e+00 1.81865335e+01 2.68467875e+01
  3.55070416e+01 4.41672956e+01 5.28275496e+01 6.14878037e+01
  7.01480577e+01 7.88083117e+01 8.74685658e+01]
 [7.66044443e-01 8.42648887e+00 1.60869333e+01 2.37473777e+01
  3.14078222e+01 3.90682666e+01 4.67287110e+01 5.43891555e+01

'dDG = np.random.randn(*dim_array)\n\nprint(dDG)\nprint(dDG.shape)\ndDG = dDG.flatten()\nprint(dDG)\nprint(dDG.shape)'

In [54]:
count = 0.
for i in range(4):
    for j in range(4):
        for k in range(4):
            count += 1
print (count)

64.0


In [55]:
test_arr = np.random.randn((*dim_array))
test_arr

array([[[-1.56236380e+00,  8.91257839e-01, -6.22286929e-01,
          5.74726757e-01],
        [ 4.68906199e-01, -1.64499136e-01,  1.64999028e-01,
         -1.30417292e+00],
        [ 9.13676972e-01, -5.25458754e-01,  8.85292132e-01,
         -5.74812157e-01],
        [-5.85320644e-01, -7.21903378e-01,  1.12723681e+00,
          1.31216079e-01]],

       [[-4.74000163e-01, -1.83417598e+00,  7.51827050e-02,
         -4.42338910e-01],
        [-6.33906962e-01, -2.40503027e-01,  1.70473139e+00,
          2.62753831e-01],
        [ 4.99682888e-01, -2.93994186e-01, -1.46584855e-01,
          2.96651813e-02],
        [-2.00367708e+00, -4.37794188e-02, -1.38481304e+00,
         -9.25769449e-01]],

       [[-7.51971912e-01,  5.41643024e-01, -2.84072193e-01,
         -1.73605626e-01],
        [-9.44300896e-01,  1.58175030e+00, -5.59727284e-01,
         -1.83521008e+00],
        [ 2.53765013e-01,  7.16124283e-01,  1.34975251e+00,
          2.24964010e+00],
        [-5.22983211e-01,  3.65489995e-

In [63]:
test_arr[0]

array([[-1.5623638 ,  0.89125784, -0.62228693,  0.57472676],
       [ 0.4689062 , -0.16449914,  0.16499903, -1.30417292],
       [ 0.91367697, -0.52545875,  0.88529213, -0.57481216],
       [-0.58532064, -0.72190338,  1.12723681,  0.13121608]])

In [67]:
test_arr[0][0,1] # -> test_arr[phi][theta,r]

0.8912578385766119

In [61]:
test_arr_1d = test_arr.flatten()
print(test_arr_1d.shape)

(64,)


In [70]:
np.ones(16).reshape(2,4,2)

array([[[1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.]],

       [[1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.]]])

In [73]:
index = np.where(test_arr > 1)
print(index)
test_arr[index]

(array([0, 1, 2, 2, 2, 3], dtype=int64), array([3, 1, 1, 2, 2, 1], dtype=int64), array([2, 2, 1, 2, 3, 3], dtype=int64))


array([1.12723681, 1.70473139, 1.5817503 , 1.34975251, 2.2496401 ,
       1.12935775])

In [74]:
# set up for loop
# run for loop from nick code
# make vectorize versions
# density = (nphi, ntheta, nr)
# density[nphi_i][ntheta_j, nr_k] is that block that represents the phi iteration;
# phi does no calculations all 
# compute the 2d array -> append to density 3d arry collect all theta 1 values

# all theta one values composed into phi block
np.log10([1,10])

array([0., 1.])